In [1]:
#r "nuget:Microsoft.ML,1.4.0-preview"
#r "nuget:Microsoft.ML.AutoML"
#r "nuget:Microsoft.Data.DataFrame,1.0.0-e190910-1"

Attempting to install package Microsoft.ML, version 1.4.0-preview............................................................done!

Successfully added reference to package Microsoft.ML, version 1.4.0-preview

Attempting to install package Microsoft.ML.AutoML............done!

Successfully added reference to package Microsoft.ML.AutoML, version 0.15.1

Attempting to install package Microsoft.Data.DataFrame, version 1.0.0-e190910-1...............done!

Successfully added reference to package Microsoft.Data.DataFrame, version 1.0.0-e190910-1

In [2]:
using Microsoft.Data;
using XPlot.Plotly;

In [3]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.RowCount); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [4]:
using System.IO;
using System.Net.Http;
string housingPath = "housing.csv";
if (!File.Exists(housingPath))
{
    var contents = new HttpClient()
        .GetStringAsync("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv").Result;
        
    File.WriteAllText("housing.csv", contents);
}

In [5]:
var housingData = DataFrame.ReadCsv(housingPath);
housingData

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY
10,-122.26,37.85,52,2202,434,910,402,3.2031,281500,NEAR BAY


In [6]:
housingData.Description()

0,Length,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,Max,500001,15.0001,6082,35682,6445,39320,52,41.95,-114.31
2,Min,14999,0.4999,1,3,0,2,1,32.54,-124.35
3,Mean,206854.97,3.8706622,499.53967,1425.4779,532.4762,2635.7588,28.639486,35.631866,-119.569115


In [7]:
Chart.Plot(
    new Graph.Histogram()
    {
        x = housingData["median_house_value"],
        nbinsx = 20
    }
)


In [8]:
var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = housingData["longitude"],
        y = housingData["latitude"],
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = housingData["median_house_value"],
            colorscale = "Jet"
        }
    }
);

chart.Width = 600;
chart.Height = 600;
display(chart);

In [9]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

In [10]:
%%time

var mlContext = new MLContext();

var experiment = mlContext.Auto().CreateRegressionExperiment(maxExperimentTimeInSeconds: 15);
var result = experiment.Execute(housingData, labelColumnName:"median_house_value");

Wall time: 15212.758000000002ms

In [11]:
var scatters = result.RunDetails.GroupBy(
    r => r.TrainerName,
    (name, details) => new Graph.Scatter()
    {
        name = name,
        x = details.Select(r => r.RuntimeInSeconds),
        y = details.Select(r => r.ValidationMetrics.MeanAbsoluteError),
        mode = "markers",
        marker = new Graph.Marker() { size = 12 }
    });

var chart = Chart.Plot(scatters);
chart.WithXTitle("Training Time");
chart.WithYTitle("Error");
display(chart);

Console.WriteLine($"Best Trainer:{result.BestRun.TrainerName}");

Best Trainer:FastTreeRegression


In [12]:
var testResults = result.BestRun.Model.Transform(housingData);

var trueValues = testResults.GetColumn<float>("median_house_value");
var predictedValues = testResults.GetColumn<float>("Score");

var predictedVsTrue = new Graph.Scatter()
{
    x = trueValues,
    y = predictedValues,
    mode = "markers",
};

var maximumValue = Math.Max(trueValues.Max(), predictedValues.Max());

var perfectLine = new Graph.Scatter()
{
    x = new[] {0, maximumValue},
    y = new[] {0, maximumValue},
    mode = "lines",
};

var chart = Chart.Plot(new[] {predictedVsTrue, perfectLine });
chart.WithXTitle("True Values");
chart.WithYTitle("Predicted Values");
chart.WithLegend(false);
chart.Width = 600;
chart.Height = 600;
display(chart);